In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.metrics import accuracy_score, classification_report,f1_score, confusion_matrix, roc_auc_score, matthews_corrcoef

## twenty percentage

In [4]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/LIME/train_20.0_lime_svm_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/LIME/test_20.0_lime_svm_selected.csv")

In [5]:
train.head(2)

,f_8,f_10,f_20,f_48,f_16,f_17,f_31,f_13,f_18,f_47,...,f_126,f_69,f_275,f_122,f_241,f_279,f_210,f_304,f_207,label
0,0.0,0.659582,0.0,0.000000,0.0,0.0,0.000000,6.078495,0.0,0.0,...,0.489515,0.0,0.0,0.0,0.0,0.0,1.301848,0.0,0.0,6.0
1,0.0,0.000000,0.0,1.826927,0.0,0.0,1.865544,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,3.0


In [6]:
x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 204)
test features shape: (700, 204)


In [7]:
# knn
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(
    n_neighbors=9,
    weights='distance',
    algorithm='auto',
    leaf_size=50
)
knn.fit(x_train, y_train)

KNeighborsClassifier(leaf_size=50, n_neighbors=9, weights='distance')

In [8]:
print("training accuracy of knn: ",knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9057142857142857


In [9]:
# svm
from sklearn.svm import SVC

svm = SVC(
    kernel='rbf',
    C=1.5,
    gamma='scale',
    probability=True,
    decision_function_shape='ovr'
)
svm.fit(x_train, y_train)

SVC(C=1.5, probability=True)

In [10]:
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

training accuracy of svm:  0.9421428571428572
test accuracy of svm 0.9171428571428571


In [11]:
# rf
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
    n_estimators=1200,
    max_depth=20,
    min_samples_leaf=1,
    random_state=42
)
rfc.fit(x_train, y_train)

RandomForestClassifier(max_depth=20, n_estimators=1200, random_state=42)

In [12]:
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

training accuracy of rfc:  1.0
test accuracy of rfc 0.9114285714285715


In [13]:
# mlp
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    alpha=0.1,
    max_iter=1000
)
mlp.fit(x_train, y_train)

MLPClassifier(alpha=0.1, max_iter=1000)

In [14]:
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of mlp:  0.9685714285714285
test accuracy of mlp 0.9157142857142857


In [15]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)

In [16]:
probs_model1_val.shape

(2800, 8)

In [17]:
# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [18]:
meta_features_train[0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       7.79770298e-03, 3.10000562e-03, 6.26015445e-03, 9.11817095e-03,
       1.08429216e-02, 4.98829508e-03, 9.55985277e-01, 1.90747203e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       6.08116045e-12, 3.91601102e-16, 6.79289809e-15, 5.92081950e-11,
       3.09288202e-14, 3.91216253e-12, 9.99999998e-01, 1.81589479e-09])

In [ ]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l2', dual=False, tol=0.0005, C=0.001, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='lbfgs', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

meta_model.fit(meta_features_train, y_train)

LogisticRegression(C=0.001, intercept_scaling=2, tol=0.0005)

In [ ]:
print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  0.9928571428571429
test accuracy of meta_model 0.9214285714285714


In [ ]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9214285714285714
MCC : 0.9099609781736503
F1 : 0.9228298726674186
AUC: 0.9906680311076556
Sensitivity: 0.9281954556393603
Specivity: 0.9721625955635352


In [ ]:

# save model
file_name = '/content/drive/MyDrive/Filters_colorectal/LIME/fs_best_20.pkl'
with open(file_name, 'wb') as model_file:
    pickle.dump(meta_model, model_file)

In [25]:
####### TAKE THIS ONE #########

# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l1', dual=False, tol=0.01, C=0.6, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='liblinear', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

meta_model.fit(meta_features_train, y_train)

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  1.0
test accuracy of meta_model 0.9128571428571428


In [26]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9128571428571428
MCC : 0.899889426930976
F1 : 0.9142152243685203
AUC: 0.9926865718128437
Sensitivity: 0.9205538484553861
Specivity: 0.9721625955635352


## forty percent

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/LIME/train_40.0_lime_svm_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/LIME/test_40.0_lime_svm_selected.csv")

train.head(2)

,f_8,f_10,f_20,f_48,f_16,f_17,f_31,f_13,f_18,f_47,...,f_370,f_400,f_465,f_426,f_422,f_397,f_467,f_130,f_464,label
0,0.0,0.659582,0.0,0.000000,0.0,0.0,0.000000,6.078495,0.0,0.0,...,0.0,0.0,0.000000,0.0,2.410424,0.0,0.0,0.0,0.000000,6.0
1,0.0,0.000000,0.0,1.826927,0.0,0.0,1.865544,0.000000,0.0,0.0,...,0.0,0.0,4.636884,0.0,0.000000,0.0,0.0,0.0,0.918282,3.0


In [ ]:
x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 409)
test features shape: (700, 409)


In [ ]:
# knn
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(
    n_neighbors=9,
    weights='distance',
    algorithm='auto',
    leaf_size=50
)
knn.fit(x_train, y_train)

KNeighborsClassifier(leaf_size=50, n_neighbors=9, weights='distance')

In [ ]:
print("training accuracy of knn: ",knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9114285714285715


In [ ]:
# svm
from sklearn.svm import SVC

svm = SVC(
    kernel='rbf',
    C=1.5,
    gamma='scale',
    probability=True,
    decision_function_shape='ovr'
)
svm.fit(x_train, y_train)

SVC(C=1.5, probability=True)

In [ ]:
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

training accuracy of svm:  0.9435714285714286
test accuracy of svm 0.9142857142857143


In [ ]:
# rf
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
    n_estimators=1200,
    max_depth=20,
    min_samples_leaf=1,
    random_state=42
)
rfc.fit(x_train, y_train)

RandomForestClassifier(max_depth=20, n_estimators=1200, random_state=42)

In [ ]:
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

training accuracy of rfc:  1.0
test accuracy of rfc 0.9128571428571428


In [ ]:
# mlp
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    alpha=0.1,
    max_iter=1000
)
mlp.fit(x_train, y_train)

MLPClassifier(alpha=0.1, max_iter=1000)

In [ ]:
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of mlp:  0.9842857142857143
test accuracy of mlp 0.9028571428571428


In [ ]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)


In [ ]:
# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [ ]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l2', dual=False, tol=0.51, C=0.01, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='saga', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

meta_model.fit(meta_features_train, y_train)

LogisticRegression(C=0.01, intercept_scaling=2, solver='saga', tol=0.51)

In [ ]:
print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  0.9971428571428571
test accuracy of meta_model 0.9157142857142857


In [ ]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9157142857142857
MCC : 0.9032294530794072
F1 : 0.9168802642050023
AUC: 0.9901926494454933
Sensitivity: 0.9231376746848392
Specivity: 0.9706959379470232


## fifty percentage

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/LIME/train_50.0_lime_svm_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/LIME/test_50.0_lime_svm_selected.csv")

x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 512)
test features shape: (700, 512)


In [ ]:
#knn
knn.fit(x_train, y_train)
print("training accuracy of knn: ", knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

# svm
svm.fit(x_train, y_train)
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

# rfc
rfc.fit(x_train, y_train)
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

# mlp
mlp.fit(x_train, y_train)
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9057142857142857
training accuracy of svm:  0.9435714285714286
test accuracy of svm 0.9185714285714286
training accuracy of rfc:  1.0
test accuracy of rfc 0.9128571428571428
training accuracy of mlp:  0.9910714285714286
test accuracy of mlp 0.9071428571428571


In [ ]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)


In [ ]:
# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [ ]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l2', dual=False, tol=0.01, C=0.01, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='saga', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

# tol=0.1, C=1.051 -> l1, liblinear
# tol=0.01, C=0.01 -> l2, newton-cg


meta_model.fit(meta_features_train, y_train)

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  0.9978571428571429
test accuracy of meta_model 0.9185714285714286


In [ ]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9185714285714286
MCC : 0.9063333936837903
F1 : 0.9193978297386496
AUC: 0.9913324122681446
Sensitivity: 0.925657182071397
Specivity: 0.9748428863594355


## sixty percent

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/LIME/train_60.0_lime_svm_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/LIME/test_60.0_lime_svm_selected.csv")

x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 614)
test features shape: (700, 614)


In [ ]:
#knn
knn.fit(x_train, y_train)
print("training accuracy of knn: ", knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

# svm
svm.fit(x_train, y_train)
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

# rfc
rfc.fit(x_train, y_train)
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

# mlp
mlp.fit(x_train, y_train)
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9071428571428571
training accuracy of svm:  0.9439285714285715
test accuracy of svm 0.9185714285714286
training accuracy of rfc:  1.0
test accuracy of rfc 0.9157142857142857
training accuracy of mlp:  0.99
test accuracy of mlp 0.9042857142857142


In [ ]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)

# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [ ]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=0.01, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='liblinear', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

# tol=0.1, C=1.051 -> l1, liblinear
# tol=0.01, C=0.01 -> l2, newton-cg
# tol=0.5, C=0.05 -> l1, saga

meta_model.fit(meta_features_train, y_train)

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  1.0
test accuracy of meta_model 0.9114285714285715


In [ ]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9114285714285715
MCC : 0.8981861161382022
F1 : 0.9126837462104092
AUC: 0.9876090550512701
Sensitivity: 0.9194613970397315
Specivity: 0.9748428863594355


## eighty percentage

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/LIME/train_80.0_lime_svm_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/LIME/test_80.0_lime_svm_selected.csv")

x_train = train.iloc[:,:-1]
x_test = test.iloc[:,:-1]

y_train = train['label']
y_test = test['label']

print(f"train features shape: {x_train.shape}")
print(f"test features shape: {x_test.shape}")

train features shape: (2800, 819)
test features shape: (700, 819)


In [ ]:
#knn
knn.fit(x_train, y_train)
print("training accuracy of knn: ", knn.score(x_train, y_train))
print("test accuracy of knn", knn.score(x_test, y_test))

# svm
svm.fit(x_train, y_train)
print("training accuracy of svm: ", svm.score(x_train, y_train))
print("test accuracy of svm", svm.score(x_test, y_test))

# rfc
rfc.fit(x_train, y_train)
print("training accuracy of rfc: ", rfc.score(x_train, y_train))
print("test accuracy of rfc", rfc.score(x_test, y_test))

# mlp
mlp.fit(x_train, y_train)
print("training accuracy of mlp: ", mlp.score(x_train, y_train))
print("test accuracy of mlp", mlp.score(x_test, y_test))

training accuracy of knn:  1.0
test accuracy of knn 0.9157142857142857
training accuracy of svm:  0.9435714285714286
test accuracy of svm 0.9214285714285714
training accuracy of rfc:  1.0
test accuracy of rfc 0.9142857142857143
training accuracy of mlp:  0.9939285714285714
test accuracy of mlp 0.9042857142857142


In [ ]:
### stacking

# train data
probs_model1_val = knn.predict_proba(x_train)
probs_model2_val = svm.predict_proba(x_train)
probs_model3_val = rfc.predict_proba(x_train)
probs_model4_val = mlp.predict_proba(x_train)

# test data
probs_model1_test = knn.predict_proba(x_test)
probs_model2_test = svm.predict_proba(x_test)
probs_model3_test = rfc.predict_proba(x_test)
probs_model4_test = mlp.predict_proba(x_test)

# meta train features
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

# meta test features
meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)

print("meta train feature shape: ", meta_features_train.shape)
print("meta test feature shape: ", meta_features_test.shape)

meta train feature shape:  (2800, 32)
meta test feature shape:  (700, 32)


In [ ]:
# classify with meta model
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(penalty='l1', dual=False, tol=0.09, C=0.08, fit_intercept=True,
                                intercept_scaling=2, class_weight=None, random_state=None,
                                solver='saga', max_iter=100, multi_class='auto', verbose=0,
                                warm_start=False, n_jobs=None, l1_ratio=None)

# tol=0.1, C=1.051 -> l1, liblinear
# tol=0.01, C=0.01 -> l2, newton-cg
# tol=0.5, C=0.05 -> l1, saga

meta_model.fit(meta_features_train, y_train)

print("training accuracy of meta_model: ", meta_model.score(meta_features_train, y_train))
print("test accuracy of meta_model", meta_model.score(meta_features_test, y_test))

training accuracy of meta_model:  1.0
test accuracy of meta_model 0.9128571428571428


In [ ]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

ACCURACY : 0.9128571428571428
MCC : 0.8998117653412029
F1 : 0.9139818984954776
AUC: 0.9899083882003676
Sensitivity: 0.9207003136017922
Specivity: 0.9721625955635352
